# 1.模型创建步骤与nn.Module

Q:LeNet的组成结构是什么？
- ![](http://anki190912.xuexihaike.com/20201001184259.png)

Q:模型构建两要素是什么？
- 1.构建子模块：`__init__()`
- 2.拼接子模块：forward()

Q:torch.nn有哪4个模块及其功能？
- nn.Parameter：张量子类，表示可学习参数，如weight, bias
- nn.Module：所有网络层基类，管理网络属性
- nn.functional：函数具体实现，如卷积，池化，激活函数等
- nn.init：参数初始化方法

Q:nn.Module中有哪些属性？
- ```python
self._parameters = OrderedDict()
self._buffers = OrderedDict()
self._backward_hooks = OrderedDict()
self._forward_hooks = OrderedDict()
self._forward_pre_hooks = OrderedDict()
self._state_dict_hooks = OrderedDict()
self._load_state_dict_pre_hooks = OrderedDict()
self._modules = OrderedDict()
```
- parameters：存储管理nn.Parameter类，如权值、偏置
- modules：存储管理nn.Module类，如卷积层、池化层
- buffers：存储管理缓冲属性，如BN层中的running_mean
- ***_hooks：存储管理钩子函数


# 2.模型容器与AlexNet构建

Q:容器(Contianers)包含哪些组件以及作用？
- nn.Sequetial：按顺序包装多个网络层
- nn.ModuleList：像python的list一样包装多个网络层
- nn.ModuleDict：像python的dict一样包装多个网络层

Q:nn.Sequential是什么？
- nn.module的容器， 用于按顺序包装一组网络层
- 顺序性：各网络层之间严格按照顺序构建
- 自带forward()：自带的forward里， 通过for循环依次执行前向传播运算



In [3]:
import torch
import torchvision
import torch.nn as nn
from collections import OrderedDict

class LeNetSequential(nn.Module):
    def __init__(self, classes):
        super(LeNetSequential, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 6, 5),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(6, 16, 5),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),)

        self.classifier = nn.Sequential(
            nn.Linear(16*5*5, 120),
            nn.ReLU(),
            nn.Linear(120, 84),
            nn.ReLU(),
            nn.Linear(84, classes),)

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size()[0], -1)
        x = self.classifier(x)
        return x


class LeNetSequentialOrderDict(nn.Module):
    def __init__(self, classes):
        super(LeNetSequentialOrderDict, self).__init__()

        self.features = nn.Sequential(OrderedDict({
            'conv1': nn.Conv2d(3, 6, 5),
            'relu1': nn.ReLU(inplace=True),
            'pool1': nn.MaxPool2d(kernel_size=2, stride=2),

            'conv2': nn.Conv2d(6, 16, 5),
            'relu2': nn.ReLU(inplace=True),
            'pool2': nn.MaxPool2d(kernel_size=2, stride=2),
        }))

        self.classifier = nn.Sequential(OrderedDict({
            'fc1': nn.Linear(16*5*5, 120),
            'relu3': nn.ReLU(),

            'fc2': nn.Linear(120, 84),
            'relu4': nn.ReLU(inplace=True),

            'fc3': nn.Linear(84, classes),
        }))

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size()[0], -1)
        x = self.classifier(x)
        return x


# net = LeNetSequential(classes=2)
net = LeNetSequentialOrderDict(classes=2)

fake_img = torch.randn((4, 3, 32, 32), dtype=torch.float32)

output = net(fake_img)

print(net)
print(output)

LeNetSequentialOrderDict(
  (features): Sequential(
    (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
    (relu1): ReLU(inplace=True)
    (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
    (relu2): ReLU(inplace=True)
    (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (fc1): Linear(in_features=400, out_features=120, bias=True)
    (relu3): ReLU()
    (fc2): Linear(in_features=120, out_features=84, bias=True)
    (relu4): ReLU(inplace=True)
    (fc3): Linear(in_features=84, out_features=2, bias=True)
  )
)
tensor([[0.0339, 0.0620],
        [0.0367, 0.0498],
        [0.0350, 0.0506],
        [0.0185, 0.0356]], grad_fn=<AddmmBackward>)


In [ ]:
Q:nn.ModuleList是什么？
- nn.module的容器，用于包装一组网络层，以迭代方式调用网络层
- 主要方法：
    - append()：在ModuleList后面添加网络层
    - extend()：拼接两个ModuleList
    - insert()：指定在ModuleList中位置插入网络层
    


In [ ]:
Q:nn.ModuleDict是什么？
- nn.module的容器，用于包装一组网络层，以索引方式调用网络层
- 主要方法：
    - clear()：清空ModuleDict
    - items()：返回可迭代的键值对 (key - value pairs)
    - keys()：返回字典的键 (key)
    - values()：返回字典的值 (value)
    - pop()：返回一对键值，并从字典中删除

# 3.nn网络层-卷积层

Q:如何对多个二维信号进行二维卷积？
- `torch.nn.quantized.Conv2d(in_channels, out_channels, kernel_size, stride=1, padding=0, dilation=1, groups=1, bias=True, padding_mode='zeros')`
- in_channels：输入通道数
- out_channels：输出通道数，等价于卷积核个数
- kernel_size：卷积核尺寸
- stride：步长
- padding：填充个数
- dilation：空洞卷积大小
- groups：分组卷积设置
- bias：偏置

Q:如何使用转置实现上采样？
- `torch.nn.ConvTranspose2d(in_channels: int, out_channels: int, kernel_size: Union[int, Tuple[int, int]], stride: Union[int, Tuple[int, int]] = 1, padding: Union[int, Tuple[int, int]] = 0, output_padding: Union[int, Tuple[int, int]] = 0, groups: int = 1, bias: bool = True, dilation: int = 1, padding_mode: str = 'zeros')`